In [1]:
import numpy as np
import sib_ldsc_z as ld
from scipy.optimize import minimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats
from importlib import reload
import matplotlib.pyplot as plt
reload(ld)

<module 'sib_ldsc_z' from 'C:\\Users\\Hariharan\\Documents\\git_repos\\SNIPar\\ldsc_reg\\inferz\\sib_ldsc_z.py'>

# Defining the PDF and the Log Likelihoods

The likelihood for a SNP $i$ is:

$$
l_i = -\frac{d}{2} log (2 \pi) - \frac{1}{2} log ( |I + r_i S_i^{-1/2} V S_i^{-1/2}| ) - \frac{1}{2} z_i^T (I + r_i S_i^{-1/2} V S_i^{-1/2}) ^{-1} z_i
$$

And its derivative:

$$
\frac{dl}{dV} = r_i S^{-1/2} \Sigma_i^{-1} (\Sigma - z_i z_i^T) \Sigma_i^{-1} S^{-1/2}
$$

In [2]:
np.random.seed(123)

# N = int(100)
# S_size = int(N/2)
# S = np.array([np.array([[.5, 0], [0, .8]]),
#     np.array([[0.5, 0], [0, 0.8]])] * S_size )
# V = np.identity(2) * 0.5
# f = np.random.uniform(0, 1, N)

N = int(1e4)
S = np.array([[[1e-4, -5 * 1e-5], [-5 * 1e-5, 1e-4]]] * N)
V = np.array([[0.5, 0.25], [0.25, 0.5]])

In [3]:
model = ld.sibreg(S = S)
model.simdata(V/N, N, simr = True)

Warning there is no value for z. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
Simulated LD scores!
Effect Vectors Simulated!


In [4]:
Vin = ld.extract_upper_triangle(V)
model.neg_logll_grad(Vin)

-38631.0386018495, [[0.5  0.25]
 [0.25 0.5 ]]


(38631.0386018495, array([-140.90639708,  -82.11590935,  125.08515464]))

In [5]:
# Testing derivatives
aderiv = model._grad_ll_v(V, model.z[0, :], model.S[0],  model.r[0])

nderiv = model._num_grad_V(V, model.z[0, :], model.S[0],  model.r[0])

np.allclose(aderiv, nderiv)

True

In [6]:
# solving
output, result = model.solve() #), gradfunc = model._num_grad_V
print(result)

No initial guess provided.
Making 'optimal' matrix
-53885.63883263604, [[1.e-06 0.e+00]
 [0.e+00 1.e-06]]
-39174.91113085403, [[0.59828088 0.55097349]
 [0.55097349 0.58179941]]
-39075.684638813575, [[0.60889756 0.53379927]
 [0.53379927 0.59148168]]
-38874.593261680006, [[0.64220394 0.47313802]
 [0.47313802 0.6182262 ]]
-38817.423705512934, [[0.65891829 0.43603959]
 [0.43603959 0.62745011]]
-38784.86161289391, [[0.6711916  0.40092216]
 [0.40092216 0.62909792]]
-38761.810937091606, [[0.67820015 0.36661315]
 [0.36661315 0.62322344]]
-38733.76991520626, [[0.67970455 0.30918831]
 [0.30918831 0.60363174]]
-38714.2543860015, [[0.66103192 0.23610379]
 [0.23610379 0.5696433 ]]
-38699.979418211406, [[0.6164181  0.19464524]
 [0.19464524 0.54090087]]
-38655.96400716247, [[0.54127437 0.2035464 ]
 [0.2035464  0.50564764]]
-38629.63701458618, [[0.50291681 0.26571183]
 [0.26571183 0.47456159]]
-38627.567327699704, [[0.52405994 0.26039677]
 [0.26039677 0.48626418]]
-38627.523595919534, [[0.52258617 0.2

In [7]:
output

array([[0.52261778, 0.26207497],
       [0.26207497, 0.48484062]])

In [8]:
modll = model._log_ll(V, model.z[0, :], model.S[0], model.r[0])

In [9]:
V_norm = V
S_inv_root = ld.calc_inv_root(model.S[0])
dist = scipy.stats.multivariate_normal(mean = None,
                                      cov = np.eye(V.shape[0]) + model.r[0] * S_inv_root @ V_norm @ S_inv_root)

nlogll = dist.logpdf(model.z[0, :])
print(nlogll)

-11.68651716929529


In [10]:
modll

-11.68651716929529

# Playing with Derivatives

In [11]:
def alogll_grad(V, S=model.S):
    
    d = S[0].shape[0]
    log_ll = 0
    Gvec = np.zeros((d, d))
    N = len(S)
    V_norm = V/N
    
    for i in range(N):
        Si = S[i]
        zi = model.z[i, :].reshape((d, 1))
        ui = model.u[i]
        ri = model.r[i]
#         Si = N * Si 


        log_ll += (1/ui) * model._log_ll(V_norm , zi, Si, ri)
        Gvec += (1/ui) * model._grad_ll_v(V_norm, zi, Si, ri)
        
    return -log_ll, -Gvec/(N)

In [12]:
def nderiv(V):
    
    g = np.zeros(V.shape)
    for i in range(0,V.shape[0]):
        for j in range(0,V.shape[1]):
            dV = np.zeros((V.shape))
            dV[i,j] = 10 ** (-6)
            V_upper = V+dV
            V_lower = V-dV
            g[i,j] = (alogll_grad(V_upper)[0] - \
                      alogll_grad(V_lower)[0]) / (2 * 10 ** (-6))
    return g


In [13]:
ajac = alogll_grad(V)[1]
print(ajac)

[[-140.90639708  -82.11590935]
 [ -82.11590935  125.08515464]]


In [14]:
njac = nderiv(V)
print(njac)

[[-140.90638797  -82.11585373]
 [ -82.11585373  125.08519285]]


In [15]:
np.allclose(njac, ajac)

True

In [16]:
alogll_grad(V)[0]

38631.0386018495

In [17]:
alogll_grad(V, S)

(38631.0386018495, array([[-140.90639708,  -82.11590935],
        [ -82.11590935,  125.08515464]]))